In [1]:
import os
from langchain_google_vertexai import ChatVertexAI

# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
model = ChatVertexAI(model_name="gemini-2.0-flash-exp",
                     temperature=0.7,
                     location="us-central1")

In [2]:
from crewai.tools import tool
from crewai_tools import ScrapeWebsiteTool
import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker: str):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example "AAPL" or "TSLA".
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Retrieve Stock news URLs")
    def stock_news(ticker: str):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker: str):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker: str):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker: str):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()


In [3]:
from crewai import Agent


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal=
            "Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory=
            "An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
            llm=model,
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal=
            "Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory=
            "You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
            llm=model,
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal=
            "Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory=
            "You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
            llm=model,
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal=
            "Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory=
            "You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
            llm=model,
        )

In [4]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description=
            "Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=
            f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description=
            "Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=
            f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def financial_analysis(self, agent):
        return Task(
            description=
            "Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=
            f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description=
            "Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=
            f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL, Entry Point, and Price Target for the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

In [5]:
from crewai import Crew
from crewai.process import Process
# from langchain_openai import ChatOpenAI
# from langchain_google_vertexai import VertexAIEmbeddings

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
financial_task = tasks.financial_analysis(financial_analyst)
technical_task = tasks.technical_analysis(technical_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager, [research_task, financial_task, technical_task])

crew = Crew(
    agents=[
        researcher, financial_analyst, technical_analyst, hedge_fund_manager
    ],
    tasks=[research_task, financial_task, technical_task, recommend_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    embedder={
        "provider": "vertexai",
        "config": {
            "model_name": "text-embedding-005",
        }
    },
)

result = crew.kickoff(inputs={"company": "Tesla motors"})

2025-01-13 15:16:17,100 - 8627864128 - rag_storage.py-rag_storage:138 - ERROR: Error during short_term search: Expected Embedings to be non-empty list or numpy array, got [] in query.
2025-01-13 15:16:18,221 - 8627864128 - rag_storage.py-rag_storage:138 - ERROR: Error during entities search: Expected Embedings to be non-empty list or numpy array, got [] in query.


# Agent: Researcher
## Task: Gather and analyze the latest news and market sentiment surrounding the stock of Tesla motors. Provide a summary of the news and any notable shifts in market sentiment.


# Agent: Researcher
## Using tool: Retrieve Stock news URLs
## Tool Input: 
"{\"ticker\": \"TSLA\"}"
## Tool Output: 
['https://finance.yahoo.com/m/c69d009d-95e5-3dd5-9d7d-f1d16852836f/investors-hope-earnings.html', 'https://finance.yahoo.com/m/32c55972-3fd9-3bc7-96d6-87f9da794209/elon-musk-to-provide-wi-fi-to.html', 'https://finance.yahoo.com/news/nvidia-ceo-jensen-huang-praises-004206796.html', 'https://finance.yahoo.com/news/europe-largest-pension-fund-sold-204323433.html', 'https://finance.yahoo.com/m/48dc8e01-d86e-3bfd-a0b6-46826c7310ec/better-buy-in-2025%3A-tesla.html', 'https://finance.yahoo.com/news/top-plays-among-the-magnificent-7-and-big-tech-stocks-as-trump-returns-to-the-white-house-162315184.html', 'https://finance.yahoo.com/m/7ff5bb63-eae2-3543-8094-50422efe9aac/the-car-mark

2025-01-13 15:23:15,501 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:23:22,678 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:23:22,873 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:23:23,601 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:23:23,795 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:23:24,525 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: E

# Agent: Financial Analyst
## Task: Analyze Tesla motors's financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.


# Agent: Financial Analyst
## Thought: Okay, I understand the task. I need to analyze Tesla's financial health and performance using the provided tools, considering the mixed market sentiment. I will start by gathering the necessary financial data, including the balance sheet, income statement, and insider trading activity. Then, I will analyze this data to assess the company's revenue, earnings, cash flow, and valuation. Finally, I will provide a detailed financial analysis report.
## Using tool: Balance sheet
## Tool Input: 
"{\"ticker\": \"TSLA\"}"
## Tool Output: 
,2023-12-31,2022-12-31,2021-12-31,2020-12-3

2025-01-13 15:25:39,219 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:25:45,343 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:25:45,536 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:25:46,261 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:25:46,463 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:25:47,184 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: E

# Agent: Technical Analyst
## Task: Conduct a detailed technical analysis of the price movements of Tesla motors's stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.


# Agent: Technical Analyst
## Using tool: One month stock price history
## Tool Input: 
"{\"ticker\": \"TSLA\"}"
## Tool Output: 
Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
2024-12-11 00:00:00-05:00,409.70001220703125,424.8800048828125,402.3800048828125,424.7699890136719,104287600,0.0,0.0
2024-12-12 00:00:00-05:00,424.8399963378906,429.29998779296875,415.0,418.1000061035156,87752200,0.0,0.0
2024-12-13 00:00:00-05:00,420.0,436.29998779296875,415.7099914550781,436.2300109863281,89000200,0.0,0.0
2024-12-16 00:00:00-05:00,441.0899963378906,463.19000244140625,436.1499938964844,463.0199890

2025-01-13 15:26:26,741 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:31,588 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:31,777 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:32,505 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:32,698 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:33,426 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: E

# Agent: Hedge Fund Manager
## Task: Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for Tesla motors's stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.


# Agent: Hedge Fund Manager
## Final Answer: 
**Investment Recommendation: BUY Tesla (TSLA)**

**Executive Summary:**

Based on a comprehensive analysis of market sentiment, financial health, and technical indicators, I recommend a **BUY** position in Tesla (TSLA). While the stock exhibits significant short-term volatility and carries inherent risks, its long-term growth potential, driven by advancements in AI and innovative product development, warrants a strategic investment. However, this recommendation comes with a strong emphasis on risk management and a phased entry approach.

**Rationale:**

1.  **Long-Term Growth Potential:**
    *   **AI Leadership:** Tesla's progress in artif

2025-01-13 15:26:47,598 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:50,702 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:50,900 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:51,625 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
2025-01-13 15:26:51,822 - 8627864128 - rag_storage.py-rag_storage:109 - ERROR: Error during entities save: Expected Embedings to be non-empty list or numpy array, got [] in add.
